In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 04:46:05,062 INFO: Initializing external client
2025-03-03 04:46:05,064 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 04:46:06,265 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215663
Fetching data from 2025-02-02 09:46:05.061832+00:00 to 2025-03-03 08:46:05.061832+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.43s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 10:00:00+00:00,2,0
1,2025-02-02 11:00:00+00:00,2,0
2,2025-02-02 12:00:00+00:00,2,0
3,2025-02-02 13:00:00+00:00,2,0
4,2025-02-02 14:00:00+00:00,2,0
...,...,...,...
174440,2025-03-03 04:00:00+00:00,263,9
174441,2025-03-03 05:00:00+00:00,263,14
174442,2025-03-03 06:00:00+00:00,263,58
174443,2025-03-03 07:00:00+00:00,263,123


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 175197
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 175197
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,2,0,0,0,0,0,0,0,1,...,0,0,0,16,2,7,0,0,255,2025-02-20 23:00:00
1,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,210,2025-02-16 19:00:00
2,3,0,0,0,0,0,1,0,2,0,...,0,0,3,0,0,0,2,1,216,2025-02-20 07:00:00
3,369,273,378,266,156,23,108,460,294,11,...,367,274,386,250,162,1,32,393,237,2025-02-17 19:00:00
4,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,147,2025-02-17 11:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,27,2025-02-07 10:00:00
247,0,3,0,1,0,0,1,1,0,1,...,0,3,0,1,2,0,1,1,62,2025-02-13 09:00:00
248,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,2,0,82,2025-02-06 03:00:00
249,41,327,100,85,245,275,23,39,122,250,...,161,73,218,81,165,19,306,149,230,2025-02-26 22:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 04:46:20,161 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 04:46:20,169 INFO: Initializing external client
2025-03-03 04:46:20,170 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 04:46:21,078 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215663
Fetching data from 2025-02-02 09:46:20.161691+00:00 to 2025-03-03 08:46:20.161691+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.07s) 


In [12]:
current_date

Timestamp('2025-03-03 09:46:20.161691+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-02 10:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,3,2025-03-02 10:00:00
2,0,4,3,2,5,0,7,2,8,1,...,65,43,15,0,1,0,1,4,4,2025-03-02 10:00:00
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-02 10:00:00
4,4,3,1,3,1,6,2,0,0,3,...,1,3,3,2,1,1,1,2,7,2025-03-02 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-02 10:00:00
247,1,2,0,3,1,0,0,1,4,0,...,0,0,0,0,2,0,0,0,260,2025-03-02 10:00:00
248,9,30,28,30,28,38,26,37,26,25,...,7,10,2,1,2,3,7,13,261,2025-03-02 10:00:00
249,85,107,91,91,76,75,59,61,46,32,...,5,2,5,8,10,25,50,74,262,2025-03-02 10:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 04:46:38,483 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 04:46:38,493 INFO: Initializing external client
2025-03-03 04:46:38,495 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 04:46:39,633 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215663

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "C:\Users\divij\anaconda3\envs\spring25_taxi\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\divij\anaconda3\envs\spring25_taxi\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\divij\anaconda3\envs\spring25_taxi\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\divij\anaconda3\envs\spring25_taxi\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,6,0.0
4,7,3.0
...,...,...
246,259,0.0
247,260,0.0
248,261,18.0
249,262,88.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 239, 237, 230, 161, 186, 142, 132, 162, 238])